In [24]:
import numpy as np
import os
import time
import cv2
#import nibabel as nib
import pdb
import matplotlib
from matplotlib import pyplot as plt
import nibabel as nib
from nibabel.testing import data_path

In [32]:
 conda install -c conda-forge jupyter_contrib_nbextensions

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - jupyter_contrib_nbextensions


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.3               |   py37h03978a9_0         3.1 MB  conda-forge
    jupyter_contrib_core-0.3.3 |             py_2          15 KB  conda-forge
    jupyter_contrib_nbextensions-0.5.1|     pyhd8ed1ab_2        19.5 MB  conda-forge
    jupyter_highlight_selected_word-0.2.0|py37h03978a9_1002          15 KB  conda-forge
    jupyter_latex_envs-1.4.6   |  pyhd8ed1ab_1002         735 KB  conda-forge
    jupyter_nbextensions_configurator-0.4.1|   py37h03978a9_2         491 KB  conda-forge
    python_abi-3.7             |          2_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           



==> WARNING: A newer version of conda exists. <==
  current version: 4.7.10
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda



EnvironmentNotWritableError: The current user does not have write permissions to the target environment.
  environment location: C:\ProgramData\Anaconda3




In [1]:
def truncated_range(img):
    max_hu = 384
    min_hu = -384
    img[np.where(img > max_hu)] = max_hu
    img[np.where(img < min_hu)] = min_hu
    return (img - min_hu) / (max_hu - min_hu) * 255.

In [19]:
path = './test/'
save_path = './data_test/data_npy/'

In [20]:
os.listdir(path)

['Patient_41.nii.gz',
 'Patient_42.nii.gz',
 'Patient_43.nii.gz',
 'Patient_44.nii.gz',
 'Patient_45.nii.gz',
 'Patient_46.nii.gz',
 'Patient_47.nii.gz',
 'Patient_48.nii.gz',
 'Patient_49.nii.gz',
 'Patient_50.nii.gz',
 'Patient_51.nii.gz',
 'Patient_52.nii.gz',
 'Patient_53.nii.gz',
 'Patient_54.nii.gz',
 'Patient_55.nii.gz',
 'Patient_56.nii.gz',
 'Patient_57.nii.gz',
 'Patient_58.nii.gz',
 'Patient_59.nii.gz',
 'Patient_60.nii.gz']

In [22]:
path = './test/'
save_path = './data_test/data_npy/'

if not os.path.exists(save_path):
    os.makedirs(save_path)

files = os.listdir(path)
count = 0
print('begin processing data')

means = []
stds = []


for i, volume in enumerate(files):
    total_imgs = []
    cur_file = os.path.join(path, volume)
    print(i, cur_file)
    cur_save_path = os.path.join(save_path, volume)
    print(cur_save_path)
    if not os.path.exists(cur_save_path):
        os.makedirs(cur_save_path)
    img = nib.load(cur_file)
    img = np.array(img.get_fdata())
    #label = nib.load(os.path.join(cur_file, 'GT.nii.gz'))
    #label = np.array(label.get_fdata())
    img = truncated_range(img)
    for idx in range(img.shape[2]):
        if idx == 0 or idx == img.shape[2] - 1:
            continue
        # 2.5D data, using adjacent 3 images
        cur_img = img[:, :, idx - 1:idx + 2].astype('uint8')
        total_imgs.append(cur_img)
        #cur_label = label[:, :, idx].astype('uint8')
        count += 1
        np.save(os.path.join(cur_save_path,volume + '_' + str(idx) + '_image.npy'), cur_img)
        #np.save(os.path.join(cur_save_path,volume + '_' + str(idx) + '_label.npy'), cur_label)
    
    total_imgs = np.stack(total_imgs, 3) / 255.
    means.append(np.mean(total_imgs))
    stds.append(np.std(total_imgs))


#print('data mean is %f' % np.mean(means))
#print('data std is %f' % np.std(stds))
#print('total data size is %f' % count)
#print('processing data end !')

begin processing data
0 ./test/Patient_41.nii.gz
./data_test/data_npy/Patient_41.nii.gz
1 ./test/Patient_42.nii.gz
./data_test/data_npy/Patient_42.nii.gz
2 ./test/Patient_43.nii.gz
./data_test/data_npy/Patient_43.nii.gz
3 ./test/Patient_44.nii.gz
./data_test/data_npy/Patient_44.nii.gz
4 ./test/Patient_45.nii.gz
./data_test/data_npy/Patient_45.nii.gz
5 ./test/Patient_46.nii.gz
./data_test/data_npy/Patient_46.nii.gz
6 ./test/Patient_47.nii.gz
./data_test/data_npy/Patient_47.nii.gz
7 ./test/Patient_48.nii.gz
./data_test/data_npy/Patient_48.nii.gz
8 ./test/Patient_49.nii.gz
./data_test/data_npy/Patient_49.nii.gz
9 ./test/Patient_50.nii.gz
./data_test/data_npy/Patient_50.nii.gz
10 ./test/Patient_51.nii.gz
./data_test/data_npy/Patient_51.nii.gz
11 ./test/Patient_52.nii.gz
./data_test/data_npy/Patient_52.nii.gz
12 ./test/Patient_53.nii.gz
./data_test/data_npy/Patient_53.nii.gz
13 ./test/Patient_54.nii.gz
./data_test/data_npy/Patient_54.nii.gz
14 ./test/Patient_55.nii.gz
./data_test/data_npy/P

In [29]:
for i in range(1,41):
    j=0
    l=1
    if(i<10):
        os.mkdir('./train_2d/Patient_0'+str(i))
        for k in os.listdir('./data/data_npy/Patient_0'+str(i)):
            j=j+1
    else:
        os.mkdir('./train_2d/Patient_'+str(i))
        for k in os.listdir('./data/data_npy/Patient_'+str(i)):
            j=j+1
    while(l<=(j/2)):
        if(i<10):
            img_array = np.load('./data/data_npy/Patient_0'+str(i)+'/Patient_0'+str(i)+'_'+str(l)+'_image.npy')
            matplotlib.image.imsave('./train_2d/Patient_0'+str(i)+'/Patient_0'+str(i)+'_'+str(l)+'_image.png', img_array)
        else:
            img_array = np.load('./data/data_npy/Patient_'+str(i)+'/Patient_'+str(i)+'_'+str(l)+'_image.npy')
            matplotlib.image.imsave('./train_2d/Patient_'+str(i)+'/Patient_'+str(i)+'_'+str(l)+'_image.png', img_array)
        l=l+1